# TODO in class
* install BeautifulSoup if you do not have it already
* pip install beautifulsoup4 
* note the 4 at the end
* find two URLs that you want to scrape on SS.com
* scrape the data from the first page
* scrape the data from the second page
* you can use the functions I provided in the class

* save both as csv files
* and make plot of the data - up to you what you want to plot


* homework will be to try to srape date from some other site - such as wikipedia

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:
url = "https://www.ss.com/en/electronics/phones/mobile-phones/apple/sell/riga_f/"
url

'https://www.ss.com/en/electronics/phones/mobile-phones/apple/sell/riga_f/'

In [23]:
req = requests.get(url)  #extracting the html
req.status_code

200

In [24]:
# parser means structuring your text into some structure
# in comes string - out comes structured data
# in this case string is req.text and structured data is a tree of html elements

soup = BeautifulSoup(req.text, 'lxml') 
soup.title

<title>SS.COM Mobile phones - Apple, Prices, Sell - Advertisements. Riga</title>

In [25]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"}) #this will work as long as html has a table row with id head_line
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]]  # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [26]:
column_names = getColList(soup)
column_names  # to be used later when creating our table

['description', 'url', 'Model', 'Volume, Gb', 'Cond.', 'Price']

In [11]:
#get all the rows from all pages
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [12]:
def getRow(row, colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [14]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist]

In [15]:
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows)
    

In [18]:
phones = "https://www.ss.com/en/electronics/phones/mobile-phones/apple/sell/riga_f/"
idf = getDFfromURL(phones)
idf.head()

,description,url,Model,"Volume, Gb",Cond.,Price
0,"Продаю Iphone XS 64gb, в хорошем состоянии. Ба...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone Xs,64,used,210 €
1,Продаю iPhone 8 в идеальном состоянии. На экра...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 8,64,new,160 €
2,"Продаю iPhone 5s 16 Gb. В хорошем состоянии , ...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone 5S,64,used,50 €
3,Состояние отличное. Три чехла в подарок. Отпеч...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 7,32,used,70 €
4,Iphone-8 plus. 100% состояние аккумулятора (но...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 8 Plus,64,used,160 €


In [19]:
idf.to_csv("phone_sell_list.csv")

In [21]:
phone = pd.read_csv('phone_sell_list.csv')
phone

,Unnamed: 0,description,url,Model,"Volume, Gb",Cond.,Price
0,0,"Продаю Iphone XS 64gb, в хорошем состоянии. Ба...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone Xs,64,used,210 €
1,1,Продаю iPhone 8 в идеальном состоянии. На экра...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 8,64,new,160 €
2,2,"Продаю iPhone 5s 16 Gb. В хорошем состоянии , ...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone 5S,64,used,50 €
3,3,Состояние отличное. Три чехла в подарок. Отпеч...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 7,32,used,70 €
4,4,Iphone-8 plus. 100% состояние аккумулятора (но...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 8 Plus,64,used,160 €
5,5,"12 Pro, 128Gb, tumši pelēks. Lietots tikai ar ...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone 12 Pro,128,used,550 €
6,6,Pārdodu Iphone 12 Pro 256 gb Grey. Pirkts pie ...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone 12 Pro,256,used,590 €
7,7,Продаю iPhone X \r\nИмеются заметные царапины ...,https://ss.com/msg/en/electronics/phones/mobil...,iPhone X,256,used,70 €
8,8,Pārdodu Apple iPhone SE 32Gb (Space Grey).Riga,https://ss.com/msg/en/electronics/phones/mobil...,iPhone SE,32,used,60 €
9,9,"Продаю iPhone SE, 32gb. Здоровье батареи 87% (...",https://ss.com/msg/en/electronics/phones/mobil...,iPhone SE,32,used,50 €
